In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [29]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"epicalyx","key":"d2e9f2d46c1176b6b93c1c5b50dc24d7"}'}

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [32]:
!kaggle datasets download -d smohapat/sg-complaint

sg-complaint.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
import pandas as pd
from zipfile import ZipFile
zip_file = ZipFile('/content/sg-complaint.zip')
fields= ['Complaint-Status','Consumer-complaint-summary'] 
data=pd.read_csv(zip_file.open('train.csv'),usecols=fields)
data.head()

,Complaint-Status,Consumer-complaint-summary
0,Closed with explanation,"Seterus, Inc a déposé un faux rapport auprès des principaux bureaux de crédit XXXX, indiquant que mon prêt hypothécaire XXXX 2015 était en retard alors qu’ils recevaient le remboursement total du prêt le XXXX XXXX, 2015. Il convient de noter que les frais de retard ne sont pas évalués avant le XXXX du chacun selon les termes et conditions de notre contrat. Je suis incapable d'obtenir une hypothèque pour ma nouvelle maison en raison de leurs actions."
1,Closed with non-monetary relief,"XX / XX / XXXX La requête en faillite n ° XXXX du chapitre XXXX a été déposée à Hawaii et incluait le créancier XXXX XXXX pour un montant inconnu (supposé {540,00 $} XXXX.\nXX / XX / XXXX Chapitre XXXX Faillite acquittée # XXXX, y compris XXXX XXXX.\nXX / XX / XXXX - XXXX XXXX a continué à transférer les créances libérées aux agences de recouvrement. XXXX a supprimé XXXX XXXX XXXX XXXX mais Experian refuse de le faire."
2,Closed with explanation,"El XXXX / XXXX / 15, estaba preparando el vuelo de regreso a XXXX (vea XXXX) para el funeral de un familiar cercano y me alojé en un hotel en el aeropuerto XXXX. Antes de irme, contraté a una cuidadora de perros y su amiga (que me habían referido personas que conocía) para que vigilaran mis perros XXXX en mi casa, recogieran mi correo y cuidaran mis plantas. Le había dado a esta persona la llave de mi casa. En XXXX / XXXX / 15, vuelo a XXXX y no regresé a XXXX hasta la tarde de XXXX / XXXX / 15. Cuando llegué a casa, me di cuenta de que mi tarjeta de débito se había utilizado para hacer aproximadamente XXXX compras desde mi cuenta de cheques con Capital One 360. Inmediatamente llamé y detuve mi tarjeta de débito, pero como había estado fuera durante más de una semana, No sabía qué había pasado. Desde que ocurrió esto, luché con Capital One con uñas y dientes, y aunque envié a Capital One una declaración jurada notariada de que los cargos XXXX no son míos, una copia de mi itinerario..."
3,Closed with explanation,"The loan was paid in XXXX XXXX. In XXXX, 4 years after I moved from VA (I moved in XXXX XX / XX / XXXX-6 months after paying the debt), a representative XXXX XXXX XXXX contacted me suddenly, indicating that my debt had not been paid and that I now owe them {$ 1100.00}. I explained that the debt was paid and, because so many years passed, I destroyed documents that I did not need to keep since the account was paid. In fact, if he owed the loan, XXXX XXXX XXXX had the right to cash the check in his possession, which he did not have because he had received it when the account was paid. That was something I asked them to provide me and they could not provide anything. Despite my letter to them and the conversation on the phone, I was continually harassed and nothing was sent to me. Then, in XXXX they contacted me through a collection letter for XXXX XXXX XXXX trying again to collect this debt. Now there are two additional years, a total of 6 years after I paid the debt that XXXX XXXX X..."
4,Closed with explanation,"J'ai obtenu un compte de crédit de soins pour XXXX. Immédiatement après avoir été facturé à tort par un autre médecin, je l'ai contacté pour me le faire savoir, au moment où il m'avait dit que j'étais responsable de cette charge et que je devais la régler moi-même et effectuer les paiements avant J'ai manqué. J'ai effectué les paiements et les frais ont été réglés par moi-même. Je n'ai jamais été crédité pour cela et j'ai facturé des intérêts et des frais de retard. Bien ce que j'ai payé, je l'ai ensuite utilisé aux fins prévues et j'ai obtenu XXXX dont j'avais besoin. Eh bien, le bureau XXXX a eu du mal à les charger car je n’ai jamais reçu de carte physique.\nJe paye donc et je reçois une notification l'informant que mon compte a été vendu à XXXX. Je les appelle pour essayer de réparer ce gâchis et ils m'envoient à une agence de recouvrement de crédits XXXX XXXX XXXX. J'appelle XXXX à plusieurs reprises au cours de 

In [34]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import LabelBinarizer

print(tf.__version__)

1.12.0


In [35]:
data = data[pd.notnull(data['Consumer-complaint-summary'])]
data.head()

,Complaint-Status,Consumer-complaint-summary
0,Closed with explanation,"Seterus, Inc a déposé un faux rapport auprès des principaux bureaux de crédit XXXX, indiquant que mon prêt hypothécaire XXXX 2015 était en retard alors qu’ils recevaient le remboursement total du prêt le XXXX XXXX, 2015. Il convient de noter que les frais de retard ne sont pas évalués avant le XXXX du chacun selon les termes et conditions de notre contrat. Je suis incapable d'obtenir une hypothèque pour ma nouvelle maison en raison de leurs actions."
1,Closed with non-monetary relief,"XX / XX / XXXX La requête en faillite n ° XXXX du chapitre XXXX a été déposée à Hawaii et incluait le créancier XXXX XXXX pour un montant inconnu (supposé {540,00 $} XXXX.\nXX / XX / XXXX Chapitre XXXX Faillite acquittée # XXXX, y compris XXXX XXXX.\nXX / XX / XXXX - XXXX XXXX a continué à transférer les créances libérées aux agences de recouvrement. XXXX a supprimé XXXX XXXX XXXX XXXX mais Experian refuse de le faire."
2,Closed with explanation,"El XXXX / XXXX / 15, estaba preparando el vuelo de regreso a XXXX (vea XXXX) para el funeral de un familiar cercano y me alojé en un hotel en el aeropuerto XXXX. Antes de irme, contraté a una cuidadora de perros y su amiga (que me habían referido personas que conocía) para que vigilaran mis perros XXXX en mi casa, recogieran mi correo y cuidaran mis plantas. Le había dado a esta persona la llave de mi casa. En XXXX / XXXX / 15, vuelo a XXXX y no regresé a XXXX hasta la tarde de XXXX / XXXX / 15. Cuando llegué a casa, me di cuenta de que mi tarjeta de débito se había utilizado para hacer aproximadamente XXXX compras desde mi cuenta de cheques con Capital One 360. Inmediatamente llamé y detuve mi tarjeta de débito, pero como había estado fuera durante más de una semana, No sabía qué había pasado. Desde que ocurrió esto, luché con Capital One con uñas y dientes, y aunque envié a Capital One una declaración jurada notariada de que los cargos XXXX no son míos, una copia de mi itinerario..."
3,Closed with explanation,"The loan was paid in XXXX XXXX. In XXXX, 4 years after I moved from VA (I moved in XXXX XX / XX / XXXX-6 months after paying the debt), a representative XXXX XXXX XXXX contacted me suddenly, indicating that my debt had not been paid and that I now owe them {$ 1100.00}. I explained that the debt was paid and, because so many years passed, I destroyed documents that I did not need to keep since the account was paid. In fact, if he owed the loan, XXXX XXXX XXXX had the right to cash the check in his possession, which he did not have because he had received it when the account was paid. That was something I asked them to provide me and they could not provide anything. Despite my letter to them and the conversation on the phone, I was continually harassed and nothing was sent to me. Then, in XXXX they contacted me through a collection letter for XXXX XXXX XXXX trying again to collect this debt. Now there are two additional years, a total of 6 years after I paid the debt that XXXX XXXX X..."
4,Closed with explanation,"J'ai obtenu un compte de crédit de soins pour XXXX. Immédiatement après avoir été facturé à tort par un autre médecin, je l'ai contacté pour me le faire savoir, au moment où il m'avait dit que j'étais responsable de cette charge et que je devais la régler moi-même et effectuer les paiements avant J'ai manqué. J'ai effectué les paiements et les frais ont été réglés par moi-même. Je n'ai jamais été crédité pour cela et j'ai facturé des intérêts et des frais de retard. Bien ce que j'ai payé, je l'ai ensuite utilisé aux fins prévues et j'ai obtenu XXXX dont j'avais besoin. Eh bien, le bureau XXXX a eu du mal à les charger car je n’ai jamais reçu de carte physique.\nJe paye donc et je reçois une notification l'informant que mon compte a été vendu à XXXX. Je les appelle pour essayer de réparer ce gâchis et ils m'envoient à une agence de recouvrement de crédits XXXX XXXX XXXX. J'appelle XXXX à plusieurs reprises au cours de 

In [36]:
pd.set_option('max_colwidth', 1000)
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.lower()
data.head()

,Complaint-Status,Consumer-complaint-summary
0,Closed with explanation,"seterus, inc a déposé un faux rapport auprès des principaux bureaux de crédit xxxx, indiquant que mon prêt hypothécaire xxxx 2015 était en retard alors qu’ils recevaient le remboursement total du prêt le xxxx xxxx, 2015. il convient de noter que les frais de retard ne sont pas évalués avant le xxxx du chacun selon les termes et conditions de notre contrat. je suis incapable d'obtenir une hypothèque pour ma nouvelle maison en raison de leurs actions."
1,Closed with non-monetary relief,"xx / xx / xxxx la requête en faillite n ° xxxx du chapitre xxxx a été déposée à hawaii et incluait le créancier xxxx xxxx pour un montant inconnu (supposé {540,00 $} xxxx.\nxx / xx / xxxx chapitre xxxx faillite acquittée # xxxx, y compris xxxx xxxx.\nxx / xx / xxxx - xxxx xxxx a continué à transférer les créances libérées aux agences de recouvrement. xxxx a supprimé xxxx xxxx xxxx xxxx mais experian refuse de le faire."
2,Closed with explanation,"el xxxx / xxxx / 15, estaba preparando el vuelo de regreso a xxxx (vea xxxx) para el funeral de un familiar cercano y me alojé en un hotel en el aeropuerto xxxx. antes de irme, contraté a una cuidadora de perros y su amiga (que me habían referido personas que conocía) para que vigilaran mis perros xxxx en mi casa, recogieran mi correo y cuidaran mis plantas. le había dado a esta persona la llave de mi casa. en xxxx / xxxx / 15, vuelo a xxxx y no regresé a xxxx hasta la tarde de xxxx / xxxx / 15. cuando llegué a casa, me di cuenta de que mi tarjeta de débito se había utilizado para hacer aproximadamente xxxx compras desde mi cuenta de cheques con capital one 360. inmediatamente llamé y detuve mi tarjeta de débito, pero como había estado fuera durante más de una semana, no sabía qué había pasado. desde que ocurrió esto, luché con capital one con uñas y dientes, y aunque envié a capital one una declaración jurada notariada de que los cargos xxxx no son míos, una copia de mi itinerario..."
3,Closed with explanation,"the loan was paid in xxxx xxxx. in xxxx, 4 years after i moved from va (i moved in xxxx xx / xx / xxxx-6 months after paying the debt), a representative xxxx xxxx xxxx contacted me suddenly, indicating that my debt had not been paid and that i now owe them {$ 1100.00}. i explained that the debt was paid and, because so many years passed, i destroyed documents that i did not need to keep since the account was paid. in fact, if he owed the loan, xxxx xxxx xxxx had the right to cash the check in his possession, which he did not have because he had received it when the account was paid. that was something i asked them to provide me and they could not provide anything. despite my letter to them and the conversation on the phone, i was continually harassed and nothing was sent to me. then, in xxxx they contacted me through a collection letter for xxxx xxxx xxxx trying again to collect this debt. now there are two additional years, a total of 6 years after i paid the debt that xxxx xxxx x..."
4,Closed with explanation,"j'ai obtenu un compte de crédit de soins pour xxxx. immédiatement après avoir été facturé à tort par un autre médecin, je l'ai contacté pour me le faire savoir, au moment où il m'avait dit que j'étais responsable de cette charge et que je devais la régler moi-même et effectuer les paiements avant j'ai manqué. j'ai effectué les paiements et les frais ont été réglés par moi-même. je n'ai jamais été crédité pour cela et j'ai facturé des intérêts et des frais de retard. bien ce que j'ai payé, je l'ai ensuite utilisé aux fins prévues et j'ai obtenu xxxx dont j'avais besoin. eh bien, le bureau xxxx a eu du mal à les charger car je n’ai jamais reçu de carte physique.\nje paye donc et je reçois une notification l'informant que mon compte a été vendu à xxxx. je les appelle pour essayer de réparer ce gâchis et ils m'envoient à une agence de recouvrement de crédits xxxx xxxx xxxx. j'appelle xxxx à plusieurs reprises au cours de 

In [37]:
import re
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('x', '')
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('{', '')
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('}', '')
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('/', '')
data.head()

,Complaint-Status,Consumer-complaint-summary
0,Closed with explanation,"seterus, inc a déposé un fau rapport auprès des principau bureau de crédit , indiquant que mon prêt hypothécaire 2015 était en retard alors qu’ils recevaient le remboursement total du prêt le , 2015. il convient de noter que les frais de retard ne sont pas évalués avant le du chacun selon les termes et conditions de notre contrat. je suis incapable d'obtenir une hypothèque pour ma nouvelle maison en raison de leurs actions."
1,Closed with non-monetary relief,"la requête en faillite n ° du chapitre a été déposée à hawaii et incluait le créancier pour un montant inconnu (supposé 540,00 $ .\n chapitre faillite acquittée # , y compris .\n - a continué à transférer les créances libérées au agences de recouvrement. a supprimé mais eperian refuse de le faire."
2,Closed with explanation,"el 15, estaba preparando el vuelo de regreso a (vea ) para el funeral de un familiar cercano y me alojé en un hotel en el aeropuerto . antes de irme, contraté a una cuidadora de perros y su amiga (que me habían referido personas que conocía) para que vigilaran mis perros en mi casa, recogieran mi correo y cuidaran mis plantas. le había dado a esta persona la llave de mi casa. en 15, vuelo a y no regresé a hasta la tarde de 15. cuando llegué a casa, me di cuenta de que mi tarjeta de débito se había utilizado para hacer aproimadamente compras desde mi cuenta de cheques con capital one 360. inmediatamente llamé y detuve mi tarjeta de débito, pero como había estado fuera durante más de una semana, no sabía qué había pasado. desde que ocurrió esto, luché con capital one con uñas y dientes, y aunque envié a capital one una declaración jurada notariada de que los cargos no son míos, una copia de mi itinerario y un informe policial, han continuado cuestionándome. todos y ..."
3,Closed with explanation,"the loan was paid in . in , 4 years after i moved from va (i moved in -6 months after paying the debt), a representative contacted me suddenly, indicating that my debt had not been paid and that i now owe them $ 1100.00. i eplained that the debt was paid and, because so many years passed, i destroyed documents that i did not need to keep since the account was paid. in fact, if he owed the loan, had the right to cash the check in his possession, which he did not have because he had received it when the account was paid. that was something i asked them to provide me and they could not provide anything. despite my letter to them and the conversation on the phone, i was continually harassed and nothing was sent to me. then, in they contacted me through a collection letter for trying again to collect this debt. now there are two additional years, a total of 6 years after i paid the debt that tries to collect for a debt that was paid in full. now , i am receiving a let..."
4,Closed with explanation,"j'ai obtenu un compte de crédit de soins pour . immédiatement après avoir été facturé à tort par un autre médecin, je l'ai contacté pour me le faire savoir, au moment où il m'avait dit que j'étais responsable de cette charge et que je devais la régler moi-même et effectuer les paiements avant j'ai manqué. j'ai effectué les paiements et les frais ont été réglés par moi-même. je n'ai jamais été crédité pour cela et j'ai facturé des intérêts et des frais de retard. bien ce que j'ai payé, je l'ai ensuite utilisé au fins prévues et j'ai obtenu dont j'avais besoin. eh bien, le bureau a eu du mal à les charger car je n’ai jamais reçu de carte physique.\nje paye donc et je reçois une notification l'informant que mon compte a été vendu à . je les appelle pour essayer de réparer ce gâchis et ils m'envoient à une agence de recouvrement de crédits . j'appelle à plusieurs reprises au cours de la journée et j'ai laissé des messages pour tenter de résoudre ce problème en vain."


In [38]:
data['Complaint-Status'].unique()

array(['Closed with explanation', 'Closed with non-monetary relief',
       'Closed', 'Closed with monetary relief', 'Untimely response'],
      dtype=object)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43266 entries, 0 to 43265
Data columns (total 2 columns):
Complaint-Status              43266 non-null object
Consumer-complaint-summary    43266 non-null object
dtypes: object(2)
memory usage: 1014.0+ KB


In [40]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43266 entries, 0 to 43265
Data columns (total 2 columns):
Complaint-Status              43266 non-null object
Consumer-complaint-summary    43266 non-null object
dtypes: object(2)
memory usage: 1014.0+ KB


In [0]:
data_comp=data[['Consumer-complaint-summary']]
data_prod=data[['Complaint-Status']]

In [0]:
train_size = int(len(data_comp) * .999)

train_descriptions = data_comp[:train_size].astype('str')
train_prod = data_prod[:train_size]

test_descriptions = data_comp[train_size:].astype('str')
test_prod =data_prod[train_size:]

In [43]:
print(train_descriptions.shape)
print(test_descriptions.shape)

(43222, 1)
(44, 1)


In [0]:
train_size = int(len(train_descriptions) * .8)

train_desc = train_descriptions[:train_size]
train_pr = train_prod[:train_size]

val_desc = train_descriptions[train_size:]
val_pr =train_prod[train_size:]

In [45]:
print(train_desc.shape)
print(val_desc.shape)

(34577, 1)
(8645, 1)


In [46]:
print(train_pr.shape)
print(val_pr.shape)

(34577, 1)
(8645, 1)


In [47]:
print(train_desc.info())
print(val_desc.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34577 entries, 0 to 34576
Data columns (total 1 columns):
Consumer-complaint-summary    34577 non-null object
dtypes: object(1)
memory usage: 540.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8645 entries, 34577 to 43221
Data columns (total 1 columns):
Consumer-complaint-summary    8645 non-null object
dtypes: object(1)
memory usage: 135.1+ KB
None


In [111]:
from sklearn import preprocessing
encoder = preprocessing.LabelBinarizer()
encoder.fit_transform(train_pr)
train_encoded = encoder.transform(train_pr)
val_encoded = encoder.transform(val_pr)
num_classes = len(encoder.classes_)

# Print all possible products and the label for the first complaint in our training dataset
print(encoder.classes_)
print(train_encoded[0])

['Closed' 'Closed with explanation' 'Closed with monetary relief'
 'Closed with non-monetary relief' 'Untimely response']
[0 1 0 0 0]


In [0]:
description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder-large/3", trainable=False)

In [0]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [52]:
features = {
  "descriptions": np.array(train_desc).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=100, num_epochs=10)
estimator = tf.contrib.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvzp43iou', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feac02cdda0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [53]:
%%timeit
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvzp43iou/model.ckpt.
INFO:tensorflow:loss = 0.69528687, step = 0
INFO:tensorflow:global_step/sec: 2.75666
INFO:tensorflow:loss = 0.2523325, step = 100 (36.282 sec)
INFO:tensorflow:global_step/sec: 2.8136
INFO:tensorflow:loss = 0.23216815, step = 200 (35.543 sec)
INFO:tensorflow:global_step/sec: 2.81181
INFO:tensorflow:loss = 0.23575777, step = 300 (35.564 sec)
INFO:tensorflow:global_step/sec: 2.806
INFO:tensorflow:loss = 0.18341379, step = 400 (35.636 sec)
INFO:tensorflow:global_step/sec: 2.81221
INFO:tensorflow:loss = 0.2585

In [55]:
%%timeit
train_input_fn_1 = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(train_desc).astype(np.str)}, train_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=train_input_fn_1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-27-09:32:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvzp43iou/model.ckpt-13832
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-27-09:34:14
INFO:tensorflow:Saving dict for global step 13832: auc = 0.9397755, auc_precision_recall = 0.802395, average_loss = 0.23121001, global_step = 13832, loss = 0.23091117
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13832: /tmp/tmpvzp43iou/model.ckpt-13832
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created becau

In [56]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(val_desc).astype(np.str)}, val_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-27-09:41:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvzp43iou/model.ckpt-13832
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-27-09:41:38
INFO:tensorflow:Saving dict for global step 13832: auc = 0.9403397, auc_precision_recall = 0.8006629, average_loss = 0.22961871, global_step = 13832, loss = 0.22939928
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13832: /tmp/tmpvzp43iou/model.ckpt-13832


{'auc': 0.9403397,
 'auc_precision_recall': 0.8006629,
 'average_loss': 0.22961871,
 'global_step': 13832,
 'loss': 0.22939928}

In [0]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, shuffle=False)

results = estimator.predict(predict_input_fn)

In [58]:
# Display predictions
for product in results:
  top = product['probabilities'].argsort()[-1:]
  for prod in top:
    text_prod = encoder.classes_[prod]
    print(text_prod + ': ' + str(round(product['probabilities'][prod] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvzp43iou/model.ckpt-13832
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Closed with explanation: 74.02%

Closed with explanation: 87.4%

Closed with explanation: 76.44%

Closed with explanation: 74.69%

Closed with explanation: 75.57%

Closed with explanation: 76.84%

Closed with explanation: 73.75%

Closed with explanation: 74.43%

Closed with explanation: 80.82%

Closed with explanation: 85.34%

Closed with explanation: 79.16%

Closed with explanation: 79.06%

Closed with explanation: 79.27%

Closed with explanation: 86.21%

Closed with explanation: 74.42%

Closed with explanation: 90.92%

Closed with explanation: 8

In [59]:
test_prod

,Complaint-Status
43222,Closed with non-monetary relief
43223,Closed with explanation
43224,Closed with explanation
43225,Closed with non-monetary relief
43226,Closed with monetary relief
43227,Closed
43228,Closed with explanation
43229,Closed with monetary relief
43230,Closed with non-monetary relief
43231,Closed with explanation


**Prediction on test data**

In [120]:
fields= ['Consumer-complaint-summary'] 
test_data=pd.read_csv(zip_file.open('test.csv'),usecols=fields)
test_data.head()


,Consumer-complaint-summary
0,"XXXX / XXXX / 16 I called Citibank to open a checking account with the promotional code of miles XXXX XXXX. I spoke with XXXX As part of the promotional fine, I had to comply with certain conditions of expense and payment of invoices. I met those, and I can provide evidence. As I did not have any proof that the promotion code applied correctly to my account, I sent a secure message to Citi to confirm it XX / XX / XXXX16. They replied that they could not see the account request to determine if a promotion code had been applied, but that if it were, they would be eligible for the promotion. Therefore, I continued to comply with the established requirements and paying a monthly fee for this checking account, based on this response. When the miles were not credited to my account, I got in touch with Citi. XX / XX / XXXX16, XXXX confirmed that a promotion code had been attached to my account, but that it was not intended for this. As Citi was unable to provide this information to me in ..."
1,I'm struggling financially. I called and I offered to pay XXXX dollars a month for my debt until I could get back up. A-1 refused to accept my payment and said they would be garnishing my salary. He even said that I would sleep well at night knowing that I would not be able to eat if I got my salary.
2,"In XXXX of 2015, an automatic payment was configured for the capital line with XXXX through our personal account XXXX, but the amount was incorrectly configured only because our first statement reflected the partial month. Our payments were made on the same day of each month in the amount of {$ 400.00} instead of {$ 680.00}, which was the correct amount. We have another mortgage with XXXX for which we have been making an additional payment of {$ 2000.00} per month during the last two years to reduce capital. The lender never notified us of the lack of payment by phone, text message or email, even though this information was provided (to me, my husband or my office), and never sent a late payment letter by mail until XX / XX / XXXX. Due to the failure of the lender to notify us properly. We have several mortgages with our businesses, lines of credit and credit cards and we have never been delayed in the last 25 years. Our credit score dropped almost XXXX points and the line closed. ..."
3,"I submitted a request to XXXX, which is my current loan provider for my student loans. The request was for the forgiveness of public loans, so XXXX advanced the request to Fed Loans Servicing in XXXX XXXX. In XXXX XXXX, I received a letter from the Fed Loans that my application was under review. I also called the Fed Loans twice in both XXXX and XXXX. They told me that my application was being reviewed and I would like to know about them and not. I'm XXXX full-time with direct loans, so I qualify for the program. My employer filled out the paperwork."
4,"A state tax lien was filed against me XXXX / XXXX / XXXX, however, I was never notified of this. Then the tax was paid and XXXX / XXXX / XXXX was released. However, I was never notified of this matter and I paid it once it appeared on my credit. However, I believed that I have 3 years to file any tax amendments if necessary. I have disputed this for several years without luck. This appears on all my credit reports."


In [0]:
test_data = test_data[pd.notnull(test_data['Consumer-complaint-summary'])]


In [122]:
test_data.shape

(18543, 1)

In [123]:
pd.set_option('max_colwidth', 1000)
test_data['Consumer-complaint-summary'] = test_data['Consumer-complaint-summary'].str.lower()
test_data.head()

,Consumer-complaint-summary
0,"xxxx / xxxx / 16 i called citibank to open a checking account with the promotional code of miles xxxx xxxx. i spoke with xxxx as part of the promotional fine, i had to comply with certain conditions of expense and payment of invoices. i met those, and i can provide evidence. as i did not have any proof that the promotion code applied correctly to my account, i sent a secure message to citi to confirm it xx / xx / xxxx16. they replied that they could not see the account request to determine if a promotion code had been applied, but that if it were, they would be eligible for the promotion. therefore, i continued to comply with the established requirements and paying a monthly fee for this checking account, based on this response. when the miles were not credited to my account, i got in touch with citi. xx / xx / xxxx16, xxxx confirmed that a promotion code had been attached to my account, but that it was not intended for this. as citi was unable to provide this information to me in ..."
1,i'm struggling financially. i called and i offered to pay xxxx dollars a month for my debt until i could get back up. a-1 refused to accept my payment and said they would be garnishing my salary. he even said that i would sleep well at night knowing that i would not be able to eat if i got my salary.
2,"in xxxx of 2015, an automatic payment was configured for the capital line with xxxx through our personal account xxxx, but the amount was incorrectly configured only because our first statement reflected the partial month. our payments were made on the same day of each month in the amount of {$ 400.00} instead of {$ 680.00}, which was the correct amount. we have another mortgage with xxxx for which we have been making an additional payment of {$ 2000.00} per month during the last two years to reduce capital. the lender never notified us of the lack of payment by phone, text message or email, even though this information was provided (to me, my husband or my office), and never sent a late payment letter by mail until xx / xx / xxxx. due to the failure of the lender to notify us properly. we have several mortgages with our businesses, lines of credit and credit cards and we have never been delayed in the last 25 years. our credit score dropped almost xxxx points and the line closed. ..."
3,"i submitted a request to xxxx, which is my current loan provider for my student loans. the request was for the forgiveness of public loans, so xxxx advanced the request to fed loans servicing in xxxx xxxx. in xxxx xxxx, i received a letter from the fed loans that my application was under review. i also called the fed loans twice in both xxxx and xxxx. they told me that my application was being reviewed and i would like to know about them and not. i'm xxxx full-time with direct loans, so i qualify for the program. my employer filled out the paperwork."
4,"a state tax lien was filed against me xxxx / xxxx / xxxx, however, i was never notified of this. then the tax was paid and xxxx / xxxx / xxxx was released. however, i was never notified of this matter and i paid it once it appeared on my credit. however, i believed that i have 3 years to file any tax amendments if necessary. i have disputed this for several years without luck. this appears on all my credit reports."


In [124]:
test_data['Consumer-complaint-summary'] = test_data['Consumer-complaint-summary'].str.replace('x', '')
test_data['Consumer-complaint-summary'] = test_data['Consumer-complaint-summary'].str.replace('{', '')
test_data['Consumer-complaint-summary'] = test_data['Consumer-complaint-summary'].str.replace('}', '')
test_data['Consumer-complaint-summary'] = test_data['Consumer-complaint-summary'].str.replace('/', '')
test_data.head()

,Consumer-complaint-summary
0,"16 i called citibank to open a checking account with the promotional code of miles . i spoke with as part of the promotional fine, i had to comply with certain conditions of epense and payment of invoices. i met those, and i can provide evidence. as i did not have any proof that the promotion code applied correctly to my account, i sent a secure message to citi to confirm it 16. they replied that they could not see the account request to determine if a promotion code had been applied, but that if it were, they would be eligible for the promotion. therefore, i continued to comply with the established requirements and paying a monthly fee for this checking account, based on this response. when the miles were not credited to my account, i got in touch with citi. 16, confirmed that a promotion code had been attached to my account, but that it was not intended for this. as citi was unable to provide this information to me in , i went ahead, and only after they fulfilled t..."
1,i'm struggling financially. i called and i offered to pay dollars a month for my debt until i could get back up. a-1 refused to accept my payment and said they would be garnishing my salary. he even said that i would sleep well at night knowing that i would not be able to eat if i got my salary.
2,"in of 2015, an automatic payment was configured for the capital line with through our personal account , but the amount was incorrectly configured only because our first statement reflected the partial month. our payments were made on the same day of each month in the amount of $ 400.00 instead of $ 680.00, which was the correct amount. we have another mortgage with for which we have been making an additional payment of $ 2000.00 per month during the last two years to reduce capital. the lender never notified us of the lack of payment by phone, tet message or email, even though this information was provided (to me, my husband or my office), and never sent a late payment letter by mail until . due to the failure of the lender to notify us properly. we have several mortgages with our businesses, lines of credit and credit cards and we have never been delayed in the last 25 years. our credit score dropped almost points and the line closed. if the lender had given us a fair cha..."
3,"i submitted a request to , which is my current loan provider for my student loans. the request was for the forgiveness of public loans, so advanced the request to fed loans servicing in . in , i received a letter from the fed loans that my application was under review. i also called the fed loans twice in both and . they told me that my application was being reviewed and i would like to know about them and not. i'm full-time with direct loans, so i qualify for the program. my employer filled out the paperwork."
4,"a state ta lien was filed against me , however, i was never notified of this. then the ta was paid and was released. however, i was never notified of this matter and i paid it once it appeared on my credit. however, i believed that i have 3 years to file any ta amendments if necessary. i have disputed this for several years without luck. this appears on all my credit reports."


In [0]:
test_data.dropna(inplace=True)

In [0]:
data_comp=test_data[['Consumer-complaint-summary']]
test_data_descriptions = data_comp[:test_data.size].astype('str')
data_prod=data[['Complaint-Status']]
test_data_prod = data_prod[:test_data.size]

In [0]:
predict_input_fn_test = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_data_descriptions).astype(np.str)}, shuffle=False)

results_test = estimator.predict(predict_input_fn_test)

In [133]:
# Display predictions

for product in results_test:
  top = product['probabilities'].argsort()[-1:]
  for prod in top:
    test_data_prod = encoder.classes_[prod]
    print(test_data_prod + ': ' + str(round(product['probabilities'][prod] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvzp43iou/model.ckpt-13832
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Closed with explanation: 61.83%

Closed with explanation: 88.52%

Closed with explanation: 64.12%

Closed with explanation: 90.45%

Closed with explanation: 86.16%

Closed with explanation: 51.97%

Closed with explanation: 85.94%

Closed with explanation: 91.66%

Closed with explanation: 82.92%

Closed with explanation: 77.81%

Closed with explanation: 74.68%

Closed with explanation: 89.07%

Closed with explanation: 93.4%

Closed with explanation: 77.94%

Closed with explanation: 85.04%

Closed with explanation: 78.77%

Closed with explanation: 8

In [134]:
test_data_prod

'Closed with explanation'